In [1]:
%matplotlib tk
%load_ext autoreload
%autoreload 2
import numpy as np
import input_fac
import matplotlib.pyplot as plt
import sk_plotting_functions as spf
import os
import json

In [8]:
el = 'Lithium'
el_sym = 'Li'
data_dir = '/Users/dpower/Documents/01 - PhD/01 - Code/11 - SIKE 2/atom_data/' + el + '/FAC/'
stages = [d for d in os.listdir(data_dir) if '_I' in d or '_V' in d or '_X' in d]
ex_trans = []; iz_trans = []; em_trans = []; rr_trans = []; ai_trans = []
uta = True
for s in stages:
    print(s)
    stage_data_dir = os.path.join(data_dir,s)
    nlj_levels = input_fac.get_levels(os.path.join(stage_data_dir, el_sym + '_en.txt'))
    ex_trans += input_fac.get_ex_cross_sections(os.path.join(stage_data_dir, el_sym + '_ce.txt'))
    iz_trans += input_fac.get_iz_cross_sections(os.path.join(stage_data_dir, el_sym + '_ci.txt'))
    em_trans += input_fac.get_em_rates(os.path.join(stage_data_dir, el_sym + '_tr.txt'), uta=uta)
    rr_trans += input_fac.get_rr_cross_sections(os.path.join(stage_data_dir, el_sym + '_rr.txt'))
    ai_trans += input_fac.get_ai_rates(os.path.join(stage_data_dir, el_sym + '_ai.txt'))
nlj_transitions = iz_trans + ex_trans + rr_trans + ai_trans + em_trans

Li_III_all
Li_I_all
Li_II_all


In [3]:
skrun = spf.SKRun('/Users/dpower/Documents/01 - PhD/14 - ELM investigation/01 - Runs/01 - Equilibria/02 - Kinetic/P_in = 4MW/Output_job_EQ_K4_2e19')
E_grid = skrun.T_norm * (skrun.vgrid / skrun.v_th) ** 2
for t in ex_trans:
    t.process_cross_section(E_grid)
for t in iz_trans:
    t.process_cross_section(E_grid)
for t in rr_trans:
    t.process_cross_section(E_grid)

In [11]:
# # Plot some transition cross-sections
# trans = iz_trans
# fig,ax = plt.subplots(1)
# for t in trans:
#     ax.plot(t.E_grid,t.sigma)
# ax.set_yscale('log')

In [7]:
# Output levels to json
with open(el_sym + '_levels_nlj.json', 'w', encoding='utf-8') as f:
    json.dump([l.__dict__ for l in nlj_levels], f, ensure_ascii=False, indent=4)

In [4]:
# Output transitions to json
for t in nlj_transitions:
    t.make_jsonable()
with open(el_sym + '_transitions_nlj.json', 'w', encoding='utf-8') as f:
    json.dump([{"E_grid": list(E_grid)}] + [(l).__dict__ for l in nlj_transitions], f, ensure_ascii=False, indent=4)


In [5]:
# Aggregate states over j
nl_levels = input_fac.aggregate_states(nlj_levels)
nl_transitions = input_fac.aggregate_transitions(nl_levels, nlj_levels, nlj_transitions)

ionization
radiative recombination
autoionization
excitation 0
excitation 1
excitation 2
excitation 3
emission 0
emission 1
emission 2
emission 3


In [6]:
# Output nl-aggregated levels and transitions to json
with open(el_sym + '_levels_nl.json', 'w', encoding='utf-8') as f:
    json.dump([l.__dict__ for l in nl_levels], f, ensure_ascii=False, indent=4)
with open(el_sym + '_transitions_nl.json', 'w', encoding='utf-8') as f:
    json.dump([{"E_grid": list(E_grid)}] + [(l).__dict__ for l in nl_transitions], f, ensure_ascii=False, indent=4)

### Some testing (to be deleted)

In [ ]:
with open('H_transitions_nl.json') as f:
    H0_trans = json.load(f)
with open('H_levels_nl.json') as f:
    H_levs = json.load(f)
with open('He_transitions_nl.json') as f:
    He1_trans = json.load(f)
with open('He_levels_nl.json') as f:
    He_levs = json.load(f)

In [ ]:
print(len(H0_trans), len(He1_trans))

In [ ]:
for i,lev in enumerate(He_levs):
    # print(i,lev['id'])
    if i != lev['id']:
        print(lev)

In [ ]:
H0_ids = []
for t in H0_trans[1:]:
    trans_id = (t['from_id'], t['to_id'])
    H0_ids.append(trans_id)
He1_ids = []
for t in He1_trans[1:]:
    trans_id = (t['from_id'], t['to_id'])
    He1_ids.append(trans_id)

In [ ]:
count = 0
for He1_id in He1_ids:
    if He1_id not in H0_ids:
        print(He1_id, 'not in H0_ids')
        count += 1
print(count)